In [25]:
import requests
import json
import pandas as pd
import numpy as np
from pandasql import sqldf

In [26]:
# Specify the stock that you want to analyze.
stock = 'TSLA'

# Request for data from Finhub.io (30 calls per second limit: https://finnhub.io/docs/api/rate-limit).
r = requests.get('https://finnhub.io/api/v1/stock/insider-transactions?symbol='+stock+'&token=c91m41iad3if9n5lk280')

# Load the JSON file as a string.
test = json.loads(r.text)

In [27]:
test

{'data': [{'symbol': 'TSLA',
   'name': 'Baglino Andrew D',
   'share': 20138,
   'change': -3500,
   'filingDate': '2022-03-30',
   'transactionDate': '2022-03-28',
   'transactionCode': 'S',
   'transactionPrice': 1065},
  {'symbol': 'TSLA',
   'name': 'Baglino Andrew D',
   'share': 23638,
   'change': 3500,
   'filingDate': '2022-03-30',
   'transactionDate': '2022-03-28',
   'transactionCode': 'M',
   'transactionPrice': 62.72},
  {'symbol': 'TSLA',
   'name': 'Baglino Andrew D',
   'share': 20138,
   'change': -897,
   'filingDate': '2022-03-08',
   'transactionDate': '2022-03-07',
   'transactionCode': 'S',
   'transactionPrice': 845.391},
  {'symbol': 'TSLA',
   'name': 'Kirkhorn Zachary',
   'share': 59318,
   'change': -2389,
   'filingDate': '2022-03-08',
   'transactionDate': '2022-03-07',
   'transactionCode': 'S',
   'transactionPrice': 845.391},
  {'symbol': 'TSLA',
   'name': 'Taneja Vaibhav',
   'share': 24684,
   'change': -1535,
   'filingDate': '2022-03-08',
   'tra

In [28]:
df = pd.DataFrame(data=test['data'])

# Derived attributes from the data.
df['dollarAmount'] = df['change']*df['transactionPrice']
df['insiderPortfolioChange'] = df['change']/(df['share'] - df['change'])

# print(type(df['transactionPrice'][0]))
conditions = [
    (df['change'] >= 0) & (df['transactionPrice'] > 0),
    (df['change'] <= 0) & (df['transactionPrice'] > 0),
    (df['transactionPrice'] == 0)
]
values = ['Buy', 'Sale', 'Gift']
df['buyOrSale'] = np.select(conditions, values)

print(df)



     symbol              name  share  change  filingDate transactionDate  \
0      TSLA  Baglino Andrew D  20138   -3500  2022-03-30      2022-03-28   
1      TSLA  Baglino Andrew D  23638    3500  2022-03-30      2022-03-28   
2      TSLA  Baglino Andrew D  20138    -897  2022-03-08      2022-03-07   
3      TSLA  Kirkhorn Zachary  59318   -2389  2022-03-08      2022-03-07   
4      TSLA    Taneja Vaibhav  24684   -1535  2022-03-08      2022-03-07   
...     ...               ...    ...     ...         ...             ...   
1236   TSLA    Taneja Vaibhav  21946    -211  2021-04-07      2021-04-05   
1237   TSLA    Taneja Vaibhav  22157     -12  2021-04-07      2021-04-05   
1238   TSLA    Taneja Vaibhav  22169    -110  2021-04-07      2021-04-05   
1239   TSLA    Taneja Vaibhav  22279    4050  2021-04-07      2021-04-05   
1240   TSLA    Taneja Vaibhav  18229      40  2021-04-07      2021-04-05   

     transactionCode  transactionPrice  dollarAmount  insiderPortfolioChange  \
0      

In [37]:
pysqldf = lambda q: sqldf(q, globals())
pysqldf("SELECT symbol, sum(change), transactionDate, transactionPrice,dollarAmount, insiderPortfolioChange, buyOrSale FROM df group by buyOrSale,transactionDate;")

,symbol,sum(change),transactionDate,transactionPrice,dollarAmount,insiderPortfolioChange,buyOrSale
0,TSLA,4090,2021-04-05,54.660,221373.000,0.222173,Buy
1,TSLA,10000,2021-04-12,55.320,553200.000,0.197636,Buy
2,TSLA,62500,2021-04-26,52.380,3273750.000,12.500000,Buy
3,TSLA,1000,2021-04-27,41.570,41570.000,0.052318,Buy
4,TSLA,10000,2021-05-10,55.320,553200.000,0.197636,Buy
...,...,...,...,...,...,...,...
84,TSLA,-1250,2022-02-17,913.260,-1141575.000,-0.021802,Sale
85,TSLA,-3500,2022-02-28,815.720,-2855020.000,-0.153853,Sale
86,TSLA,-25000,2022-03-01,886.227,-265868.100,-0.056604,Sale
87,TSLA,-4821,2022-03-07,845.391,-758315.727,-0.042643,Sale


In [40]:

sales = pysqldf("SELECT symbol, sum(change), transactionDate, sum(transactionPrice), sum(dollarAmount), insiderPortfolioChange, count(buyOrSale) FROM df where buyOrSale == 'Sale' group by transactionDate;")

In [41]:

buys = pysqldf("SELECT symbol, sum(change), transactionDate, sum(transactionPrice), sum(dollarAmount), insiderPortfolioChange, count(buyOrSale) FROM df where buyOrSale == 'Buy' group by transactionDate;")

In [46]:

allVals = pysqldf("SELECT symbol, sum(change), transactionDate, sum(transactionPrice), sum(dollarAmount), insiderPortfolioChange, buyOrSale,count(buyOrSale) FROM df  group by buyOrSale,transactionDate;")

In [48]:
df[['transactionDate']]

,transactionDate
0,2022-03-28
1,2022-03-28
2,2022-03-07
3,2022-03-07
4,2022-03-07
...,...
1236,2021-04-05
1237,2021-04-05
1238,2021-04-05
1239,2021-04-05


In [14]:
#df.to_csv('tesla_insider.csv', index=False) 

In [12]:
# Specify the stock that you want to analyze.
stock = 'GOOG'

# Request for data from Finhub.io (30 calls per second limit: https://finnhub.io/docs/api/rate-limit).
r = requests.get('https://finnhub.io/api/v1/stock/insider-transactions?symbol='+stock+'&token=c91m41iad3if9n5lk280')

# Load the JSON file as a string.
test2 = json.loads(r.text)

In [13]:
test2

{'data': [{'symbol': 'GOOG',
   'name': "O'Toole Amie Thuener",
   'share': 1159,
   'change': -42,
   'filingDate': '2022-04-01',
   'transactionDate': '2022-04-01',
   'transactionCode': 'S',
   'transactionPrice': 2800.2},
  {'symbol': 'GOOG',
   'name': "O'Toole Amie Thuener",
   'share': 953,
   'change': -53,
   'filingDate': '2022-03-29',
   'transactionDate': '2022-03-25',
   'transactionCode': 'C',
   'transactionPrice': 0},
  {'symbol': 'GOOG',
   'name': "O'Toole Amie Thuener",
   'share': 1006,
   'change': -53,
   'filingDate': '2022-03-29',
   'transactionDate': '2022-03-25',
   'transactionCode': 'F',
   'transactionPrice': 2826.24},
  {'symbol': 'GOOG',
   'name': "O'Toole Amie Thuener",
   'share': 1201,
   'change': 53,
   'filingDate': '2022-03-29',
   'transactionDate': '2022-03-25',
   'transactionCode': 'C',
   'transactionPrice': 0},
  {'symbol': 'GOOG',
   'name': 'Schindler Philipp',
   'share': 7238,
   'change': -385,
   'filingDate': '2022-03-29',
   'transa

In [15]:
df = pd.DataFrame(data=test2['data'])

# Derived attributes from the data.
df['dollarAmount'] = df['change']*df['transactionPrice']
df['insiderPortfolioChange'] = df['change']/(df['share'] - df['change'])

# print(type(df['transactionPrice'][0]))
conditions = [
    (df['change'] >= 0) & (df['transactionPrice'] > 0),
    (df['change'] <= 0) & (df['transactionPrice'] > 0),
    (df['transactionPrice'] == 0)
]
values = ['Buy', 'Sale', 'Gift']
df['buyOrSale'] = np.select(conditions, values)

print(df)

     symbol                  name     share  change  filingDate  \
0      GOOG  O'Toole Amie Thuener      1159     -42  2022-04-01   
1      GOOG  O'Toole Amie Thuener       953     -53  2022-03-29   
2      GOOG  O'Toole Amie Thuener      1006     -53  2022-03-29   
3      GOOG  O'Toole Amie Thuener      1201      53  2022-03-29   
4      GOOG     Schindler Philipp      7238    -385  2022-03-29   
...     ...                   ...       ...     ...         ...   
1995   GOOG         Page Lawrence  19704031   -1200  2021-12-09   
1996   GOOG         Page Lawrence  19705231    -930  2021-12-09   
1997   GOOG         Page Lawrence  19706161    -700  2021-12-09   
1998   GOOG         Page Lawrence  19706861    -400  2021-12-09   
1999   GOOG         Page Lawrence  19707261    -197  2021-12-09   

     transactionDate transactionCode  transactionPrice  dollarAmount  \
0         2022-04-01               S         2800.2000 -1.176084e+05   
1         2022-03-25               C            0.0

In [16]:
df[['transactionDate']]

,transactionDate
0,2022-04-01
1,2022-03-25
2,2022-03-25
3,2022-03-25
4,2022-03-25
...,...
1995,2021-12-08
1996,2021-12-08
1997,2021-12-08
1998,2021-12-08


In [17]:
df.to_csv('google_insider.csv', index=False) 

In [18]:
# Specify the stock that you want to analyze.
stock = 'WFC'
#much less data fro this one
# Request for data from Finhub.io (30 calls per second limit: https://finnhub.io/docs/api/rate-limit).
r = requests.get('https://finnhub.io/api/v1/stock/insider-transactions?symbol='+stock+'&token=c91m41iad3if9n5lk280')

# Load the JSON file as a string.
test3 = json.loads(r.text)

In [19]:
df = pd.DataFrame(data=test3['data'])

# Derived attributes from the data.
df['dollarAmount'] = df['change']*df['transactionPrice']
df['insiderPortfolioChange'] = df['change']/(df['share'] - df['change'])

# print(type(df['transactionPrice'][0]))
conditions = [
    (df['change'] >= 0) & (df['transactionPrice'] > 0),
    (df['change'] <= 0) & (df['transactionPrice'] > 0),
    (df['transactionPrice'] == 0)
]
values = ['Buy', 'Sale', 'Gift']
df['buyOrSale'] = np.select(conditions, values)

print(df)

   symbol              name  share  change  filingDate transactionDate  \
0     WFC  Van Beurden Saul  54492   -3837  2022-03-17      2022-03-15   
1     WFC  Van Beurden Saul  58329    7516  2022-03-17      2022-03-15   
2     WFC  Van Beurden Saul  50813   -5701  2022-03-17      2022-03-15   
3     WFC  Van Beurden Saul  56514   11166  2022-03-17      2022-03-15   
4     WFC      Powell Scott  73836    -879  2022-03-17      2022-03-15   
..    ...               ...    ...     ...         ...             ...   
95    WFC  Van Beurden Saul  52555   30772  2021-04-09      2021-04-08   
96    WFC  Van Beurden Saul  21783   -1804  2021-04-09      2021-04-08   
97    WFC  Van Beurden Saul  23587    3663  2021-04-09      2021-04-08   
98    WFC  Flowers Derek A.  10657   -8615  2021-04-09      2021-04-07   
99    WFC  Flowers Derek A.  19272   19272  2021-04-09      2021-04-07   

   transactionCode  transactionPrice  dollarAmount  insiderPortfolioChange  \
0                F             50

In [ ]:
#no csv because less data. do nto use in project

